In [40]:
import urllib
import json
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import random

def get_data():
    lb_url = 'http://www.golfchannel.com/tours/pga-tour/2017/quicken-loans-national/'
    lb_url2 = 'http://www.golfchannel.com/tours/pga-tour/2017/fedex-st-jude-classic/'
    r = requests.get(lb_url)
    soup = BeautifulSoup(r.content, 'html.parser')

    stuff = soup.findAll('script')[25].get_text().replace('\n    var tournamentJSON = ', '') \
                    .replace(';\n  $(\'body\').data(\'tournamentJSON\', tournamentJSON);\n', '')
    data = json.loads(stuff) 
    print(data)
    scorecard = []
    # get scorecard
    for key in data['courses']:
        course = key

    front_yard = 0
    front_par = 0
    back_par = 0
    back_yard = 0
    for i in range(1, 19):
        new = {}
        new['hole'] = str(i)
        new['par'] = int(data['courses'][course]['holes'][str(i)]['par'])
        new['yardage'] = int(data['courses'][course]['holes'][str(i)]['yards'])
        scorecard.append(new)
        if i <= 9:
            front_yard = front_yard + new['yardage']
            front_par = front_par + new['par']
        if i == 9:
            new= {}
            new['hole'] = 'OUT'
            new['par'] = front_par
            new['yardage'] = front_yard
            scorecard.append(new)
        if i >= 10:
            back_yard = back_yard + new['yardage']
            back_par = back_par + new['par']
        if i == 18:
            new= {}
            new['hole'] = 'IN'
            new['par'] = back_par
            new['yardage'] = back_yard
            scorecard.append(new)
            new = {}
            new['hole'] = 'TOT'
            new['par'] = back_par + front_par
            new['yardage'] = back_yard + front_yard
            scorecard.append(new)
        
    
    scorecard = pd.DataFrame(scorecard)

    all_scores = []
    board = []
    scores = data['scoreboard']['players']
    for p_num in scores:
        today = '-'
        if len(scores[p_num]['score_cards']) == 0:
            new = {}
            new['name'] = scores[p_num]['firstname'] + ' ' + scores[p_num]['lastname']
            new['round'] = '1'
            for i in range(1 , 19):
                new['hole'+str(i)] = np.nan
            new['OUT'] = np.nan
            new['IN'] = np.nan
            new['TOTAL'] = np.nan
            all_scores.append(new)
        else:
            for key in scores[p_num]['score_cards']:
                if int(key) == len(scores[p_num]['score_cards']):
                    thru = 18 - scores[p_num]['score_cards'][str(key)]['round_values'].count('')
                    if thru > 0:
                        today = 0
                        for i in range(1, 19):
                            if scores[p_num]['score_cards'][str(key)]['round_values'][i-1] == '':
                                continue
                            else:
                                today = today + int(scores[p_num]['score_cards'][str(key)]['round_values'][i-1]) - \
                                            int(scorecard.loc[scorecard.hole == str(i), 'par'].max())
                        if today == 0:
                            today = 'E'
                        elif today > 0:
                            today = '+' + str(today)
                        else:
                            today = str(today)
                    else:
                        today = '-'
                    if thru == 18:
                        thru = 'F'

                new = {}
                new['name'] = scores[p_num]['firstname'] + ' ' + scores[p_num]['lastname']
                new['round'] = key
                new['morning'] = int(round(random.uniform(0,1), 0))
                out = 0
                inn = 0
                total = 0
                for i in range(1, 19):
                    try:
                        new['hole'+str(i)] = int(scores[p_num]['score_cards'][str(key)]['round_values'][i-1])
                        if i <= 9:
                            out = out + new['hole'+str(i)]
                        else:
                            inn = inn + new['hole'+str(i)]
                    except:
                        if i <= 9:
                            out = np.nan
                        else:
                            inn = np.nan
                        new['hole'+str(i)] = np.nan
                new['OUT'] = out
                new['IN'] = inn
                new['TOTAL'] = out + inn
                all_scores.append(new)
        
        

        new = {}
        new['position'] = scores[p_num]['pos']
        new['change'] = scores[p_num]['rank_change']
        new['change_type'] = scores[p_num]['rank_change_class']
        new['name'] = scores[p_num]['firstname'] + ' ' + scores[p_num]['lastname']
        new['total_score'] = scores[p_num]['scores']['Total']
        new['R1'] = scores[p_num]['scores']['R1']
        new['R2'] = scores[p_num]['scores']['R2']
        new['R3'] = scores[p_num]['scores']['R3']
        new['R4'] = scores[p_num]['scores']['R4']
        try:
            thru = scores[p_num]['scores']['TeeTime']
            new['thru'] = thru
            new['today'] = ''
        except:
            new['thru'] = thru
            new['today'] = today
        board.append(new)
        
    all_scores = pd.DataFrame(all_scores)
    board = pd.DataFrame(board)
    board.loc[board['position'].isin(['WD', 'DQ', 'CUT']), 'thru'] = '-'
    board.loc[board['position'].isin(['WD', 'DQ', 'CUT']), 'today'] = '-'


    scorecard['avg_1'] = 'None'
    scorecard['avg_2'] = 'None'
    scorecard['avg_3'] = 'None'
    scorecard['avg_4'] = 'None'
    scorecard['avg_this'] = 'None'
    scorecard['avg_total'] = 'None'
    for i in range(1, 19):
        try:
            hole_avg = all_scores['hole'+str(i)].mean()
            scorecard.loc[scorecard['hole'] == str(i), 'avg_total'] = hole_avg
            hole_avg_this = all_scores.loc[all_scores['round'] == all_scores['round'].max(), 'hole'+str(i)].mean()
            scorecard.loc[scorecard['hole'] == str(i), 'avg_this'] = hole_avg_this
            for j in range(1, int(all_scores['round'].max()) + 1):
                hole_avg = all_scores['hole'+str(i)][all_scores['round'] == str(j)].mean()
                hole_avg_morn = all_scores['hole'+str(i)][(all_scores['round'] == str(j)) &
                                                         (all_scores['morning'] == 1)].mean()
                hole_avg_aft = all_scores['hole'+str(i)][(all_scores['round'] == str(j)) &
                                                         (all_scores['morning'] == 0)].mean()
                scorecard.loc[scorecard['hole'] == str(i), 'avg_'+str(j)] = hole_avg
                scorecard.loc[scorecard['hole'] == str(i), 'morn_avg_'+str(j)] = hole_avg_morn
                scorecard.loc[scorecard['hole'] == str(i), 'aft_avg_'+str(j)] = hole_avg_aft

        except:
            print('nobody has played hole ' + str(i))
    # only if yardages are zero!!        
    scorecard['yardage'] = [597,360,515,456,505,208,607,492,135,3875, 525,468,464,193,594,357,183,509,637,3930, 7805]
    
    board = board[['position', 'change', 'change_type', 'name', 'total_score', 'thru', 'today', 
              'R1', 'R2', 'R3', 'R4']]
    all_scores = all_scores[['name','round','hole1',
           'hole2',
           'hole3',
           'hole4',
           'hole5',
           'hole6',
           'hole7',
           'hole8',
           'hole9',
           'OUT',
           'hole10',
           'hole11',
           'hole12',
           'hole13',
           'hole14',
           'hole15',
           'hole16',
           'hole17',
           'hole18',
           'IN',
           'TOTAL']]

    all_scores_sg = all_scores.copy()
    sg = pd.DataFrame()
    for r in range(1, int(all_scores['round'].max()) + 1):
        current = all_scores_sg[all_scores_sg['round'] == str(r)]
        for hole in range(1, 19):
            current['hole'+str(hole)] = scorecard.loc[scorecard['hole'] == str(hole), 'avg_'+str(r)].mean() - current['hole'+str(hole)]
        sg = sg.append(current)
    
    sg['OUT'] = sg['hole1'] + sg['hole2'] + sg['hole3'] + sg['hole4'] + sg['hole5'] + sg['hole6'] + \
            sg['hole7'] + sg['hole8'] + sg['hole9']
    sg['IN'] = sg['hole10'] + sg['hole11'] + sg['hole12'] + sg['hole13'] + sg['hole14'] + sg['hole15'] + \
            sg['hole16'] + sg['hole17'] + sg['hole18']
    sg['TOTAL'] = sg['IN'] + sg['OUT']  

    sg = sg.fillna(0)
    sg = sg.sort_values('round', ascending=False)

    sg['type'] = 'SG'
    all_scores['type'] = 'Scores'
    all_scores = all_scores.append(sg)
    all_scores = all_scores.sort_values('round', ascending=False)

    scorecard2 = scorecard.loc[(scorecard['hole'] != 'IN') &
            (scorecard['hole'] != 'OUT') &
             (scorecard['hole'] != 'TOT')]

    # need to at least have round 1 so change those Nones to 0s
    scorecard2.loc[scorecard['avg_1'] == 'None', 'avg_1'] = 0
    
    # figure out what the current round is
    scorecard2['current_round'] = 1
    scorecard2['total_avg_ready'] = 0
    for i in range(2, 5):
        if len(scorecard2[scorecard2['avg_' + str(i)] == 'None']) != 18:
            scorecard2['current_round'] = i
            scorecard2.loc[scorecard2['avg_'+str(i)] == 'None', 'avg_' + str(i)] = 0
            scorecard2.loc[pd.isnull(scorecard2['avg_'+str(i)]), 'avg_' + str(i)] = 0
        else:
            scorecard2.loc[scorecard2['avg_'+str(i)] == 'None', 'avg_' + str(i)] = 0
        if len(scorecard2[scorecard2['avg_' + str(i)].isin([0, 'None'])]) == 0:
            scorecard2['total_avg_ready'] = i

    scorecard2['rel_max'] = max((scorecard2['avg_1'].astype(float) - scorecard2['par'].astype(float)).max(),
                   (scorecard2['avg_2'].astype(float) - scorecard2['par'].astype(float)).max(),
                   (scorecard2['avg_3'].astype(float) - scorecard2['par'].astype(float)).max(),
                   (scorecard2['avg_4'].astype(float) - scorecard2['par'].astype(float)).max(),                   
                  )
    min_card = scorecard2.replace(0, 100)
    scorecard2['rel_min'] = min((min_card['avg_1'].astype(float) - min_card['par'].astype(float)).min(),
                   (min_card['avg_2'].astype(float) - min_card['par'].astype(float)).min(),
                   (min_card['avg_3'].astype(float) - min_card['par'].astype(float)).min(),
                   (min_card['avg_4'].astype(float) - min_card['par'].astype(float)).min(),                   
                  )
    scorecard2['max'] = max(scorecard2['avg_1'].astype(float).max(),
                   scorecard2['avg_2'].astype(float).max(),
                   scorecard2['avg_3'].astype(float).max(),
                   scorecard2['avg_4'].astype(float).max(),                   
                    )
    # get max and min for relative and 

    scorecard2['course_name'] = data['courses'][course]['name']
    scorecard2['event_name'] = data['title']
    scorecard2['course_par'] = data['par'][course]
    print(scorecard2.head())
    all_scores.to_csv('~/Dropbox/scorecards.csv', index=False)
    #sg.to_csv('~/Dropbox/sg_card.csv', index=False)
    board.to_csv('~/Dropbox/gc_board.csv', index=False)
    scorecard.to_csv('~/Dropbox/sc.csv', index=False)
    scorecard2.to_csv('~/Dropbox/sc2.csv', index=False)

i = 1
while i <= 1:
    try:
        get_data()
    except:
        print('oops!')
    # sleep 30 seconds
    #time.sleep(180)
    i += 1

{'courses': {'1391': {'id': '1391', 'holes': {'1': {'number': '1', 'par': '4', 'scores': [], 'yards': '440'}, '2': {'number': '2', 'par': '5', 'scores': [], 'yards': '619'}, '3': {'number': '3', 'par': '3', 'scores': [], 'yards': '225'}, '4': {'number': '4', 'par': '4', 'scores': [], 'yards': '440'}, '5': {'number': '5', 'par': '4', 'scores': [], 'yards': '365'}, '6': {'number': '6', 'par': '4', 'scores': [], 'yards': '484'}, '7': {'number': '7', 'par': '4', 'scores': [], 'yards': '452'}, '8': {'number': '8', 'par': '4', 'scores': [], 'yards': '467'}, '9': {'number': '9', 'par': '3', 'scores': [], 'yards': '201'}, '10': {'number': '10', 'par': '5', 'scores': [], 'yards': '560'}, '11': {'number': '11', 'par': '4', 'scores': [], 'yards': '470'}, '12': {'number': '12', 'par': '3', 'scores': [], 'yards': '168'}, '13': {'number': '13', 'par': '4', 'scores': [], 'yards': '360'}, '14': {'number': '14', 'par': '4', 'scores': [], 'yards': '299'}, '15': {'number': '15', 'par': '4', 'scores': [],

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

  hole  par  yardage    avg_1    avg_2 avg_3 avg_4 avg_this avg_total  \
0    1    4      597  4.04202  4.12821     0     0  4.12821   4.08475   
1    2    5      360  4.98319  4.81197     0     0  4.81197   4.89831   
2    3    3      515  3.19328  3.12821     0     0  3.12821   3.16102   
3    4    4      456  4.07563  4.40171     0     0  4.40171   4.23729   
4    5    4      505   4.0084  3.93162     0     0  3.93162   3.97034   

   morn_avg_1    ...      morn_avg_2  aft_avg_2  current_round  \
0    3.851852    ...        4.107143   4.147541              2   
1    4.944444    ...        4.767857   4.852459              2   
2    3.240741    ...        3.053571   3.196721              2   
3    3.925926    ...        4.303571   4.491803              2   
4    4.055556    ...        3.982143   3.885246              2   

   total_avg_ready   rel_max   rel_min       max                 course_name  \
0                2  0.543103 -0.403361  5.008403  TPC Potomac at Avenel Farm   
1   

In [39]:
int(round(random.uniform(0,1), 0))

1